In [ ]:
#Création matrices movies similarities et user similarities

In [1]:
import pandas as pd
import numpy as np
from time import time
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from scipy import sparse
from tqdm import tqdm

In [2]:
ratings=pd.read_csv('../../../dataset/ml-20m/ratings.csv')
df_ref=pd.read_csv('../../new_dataset/correspondances_Id_movie')

In [3]:
# Ajout de movieId_ref dans fichier ratings, et suppression infos inutiles
ratings.index=ratings.movieId
df_ref.index=df_ref.movieId
ratings=ratings.join(df_ref.movieId_ref)
ratings=ratings.drop(["timestamp", "movieId"], axis=1)
ratings=ratings.dropna(subset=["movieId_ref"])
print(len(ratings.movieId_ref.unique()))
ratings.movieId_ref=ratings.movieId_ref.astype(int)
ratings.head()
del df_ref

23608


In [4]:
#séparer jeu de test jeu d entrainement
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=222)

In [5]:
#Définition du nombre de users et du nombre de films

n_users = ratings.userId.unique().shape[0]
n_movies = ratings.movieId_ref.unique().shape[0]
print ("Nombre d'utilisateurs : " + str(n_users) + " | Nombre de films : " + str(n_movies))
del ratings

Nombre d'utilisateurs : 138493 | Nombre de films : 23608


In [6]:
train_data.head()

,userId,rating,movieId_ref
movieId,,,
1682,4563,3.0,1567
52,21601,3.0,50
110,57184,5.0,104
724,26937,3.0,682
39183,118529,5.0,9973


In [18]:
# Création de 2 matrices modèles train et test

matrice_train = sparse.dok_matrix((n_users, n_movies), dtype=np.int8)
for line in tqdm(train_data.itertuples()):
    matrice_train[(line[1]-1, line[3]-1)] = line[2]

15577789it [02:37, 99055.62it/s] 


In [19]:
del train_data
del test_data
del n_users
del n_movies

In [21]:
# Création de matrice de similarité movies

t0=time()
movie_similarity = cosine_similarity(matrice_train.T)
t1=time()

print("temps : ",t1-t0)

('temps : ', 62.90984392166138)


In [22]:
#Enregistrement matrice similarité movies
np.savetxt('../../../dataset/movie_similarity.csv', movie_similarity, fmt='%10.5f')
del movie_similarity

In [23]:
# Création de matrice de similarité users

user_similarity=cosine_similarity(matrice_train)

In [ ]:
#Enregistrement matrice similarité users
np.savetxt('../../../dataset/user_similarity.csv', user_similarity, fmt='%10.5f')
del movie_similarity

In [10]:
#Enregistrement matrice creuse train
np.savetxt('../../../dataset/matrice_model_train_creuse.csv', matrice_train_s, fmt='%10.5f')